In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

CONNECT_INDY = False
CONNECT_MOBILE = False
USE_SDK = True

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001
WS_HEIGHT = 1.6
COL_COLOR = (1,1,1,0.2)

from pkg.controller.combined_robot import *
from pkg.project_config import *

if not CONNECT_INDY:
    indy_7dof_client.kiro_tool.OFFLINE_MODE = True
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True} if not USE_SDK else {})

ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

connection command:
kmb0: True
indy1: False


### Make scene

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.planning.scene import PlanningScene

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)

gscene.set_workspace_boundary(
    -1, 3, -2, 2, -CLEARANCE, WS_HEIGHT, thickness=WALL_THICKNESS)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


### Planning scene

In [4]:
from pkg.geometry.geometry import *
from pkg.utils.code_scraps import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [5]:
pscene = PlanningScene(gscene, combined_robot=crob)
ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
CAM_LINK = TIP_LINK.replace("tcp", "link6")
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE
BRUSH_NAME = "brush_face"
brush_face = add_indy_sweep_tool(gscene, ROBOT_NAME, face_name=BRUSH_NAME, tool_offset=0.12, color_col=(0.8,0.8,0.8,0.2))
brush_b = pscene.create_binder(BRUSH_NAME, BRUSH_NAME, WayFramer, point=(0,0,brush_face.dims[2]/2), rpy=(0,np.pi,0))
# from demo_utils.environment import add_kiro_indytool_down
# add_kiro_indytool_down(gscene, zoff=0, tool_link="indy1_tcp", face_name="brush_face", ext_off=0.032, tool_dim=(0.08, 0.32))
gscene.show_pose(crob.home_pose)

MOBILE_PUSH_NAME = "mpush"
MOBILE_PUSH_HEIGHT = 0.4
MOBILE_PUSH_KEY = 1
front_dist = 0.45+CLEARANCE
mpush_g = gscene.create_safe(GEOTYPE.SPHERE, MOBILE_PUSH_NAME, link_name=HOLD_LINK,dims=(0.01,0.01,0.01),
                           center=(front_dist, 0, MOBILE_PUSH_HEIGHT), rpy=(0,np.pi/2,0),
                           fixed=True, collision=False, color=(1,0,0,0.5))
mpush_b = pscene.create_binder(MOBILE_PUSH_NAME, MOBILE_PUSH_NAME, FramedTool, point=(0,0,0), rpy=(0,0,0), key=MOBILE_PUSH_KEY)

floor_b = pscene.create_binder("floor_ws", "floor_ws", PlaceFrame, point=(0,0,WALL_THICKNESS/2), key=MOBILE_PUSH_KEY)

TOOL_PUSH_NAME = "tpush"
TOOL_PUSH_HEIGHT = 0.03
TOOL_PUSH_KEY = 10
brush_col = gscene.NAME_DICT[BRUSH_NAME+"_col"]
TOOL_DIM = brush_col.dims
tpush_g = gscene.create_safe(GEOTYPE.SPHERE, TOOL_PUSH_NAME, link_name=TIP_LINK,dims=(0.01,0.01,0.01),
                           center=(-TOOL_DIM[0]/2-CLEARANCE, 0, 0), rpy=(0,-np.pi/2,0),
                           fixed=True, collision=False, color=(1,0,0,0.5), parent=BRUSH_NAME)
tpush_b = pscene.create_binder(TOOL_PUSH_NAME, TOOL_PUSH_NAME, FramedTool, point=(0,0,0), rpy=(0,0,0), key=TOOL_PUSH_KEY)


## Table

In [6]:
TABLE_DIM = (0.6, 1.2, 0.7)
TABLE_NAME = "table"
TABLE_LOC = (2, 0, TABLE_DIM[2]/2)
TABLE_RPY = (0,0,0)

In [7]:
table_g = gscene.create_safe(GEOTYPE.BOX, TABLE_NAME, link_name="base_link", dims=TABLE_DIM,
                             center=TABLE_LOC, rpy=TABLE_RPY,
                             fixed=True, collision=True, color=(0.8,0.8,0.8,1))
table_b = pscene.create_binder(TABLE_NAME, TABLE_NAME, PlaceFrame, point=(0,0,TABLE_DIM[2]/2), key=TOOL_PUSH_KEY)

## Waypoints

In [8]:
WP_NAME = "wp_{}"
WP_DIM = (0.1,0.1,0.01)
WP_LOC = (-0.2, 0.0, TABLE_DIM[2]/2+WP_DIM[2]/2)
WP_RPY = (0,0,0)

In [9]:
wp_g = gscene.create_safe(GEOTYPE.BOX, WP_NAME.format(0), link_name="base_link", dims=WP_DIM,
                          center=WP_LOC, rpy=WP_RPY,
                          fixed=True, collision=True, color=(0.0,0.0,0.8,1), parent=TABLE_NAME)
waypoint_s = pscene.create_subject(oname="waypoints", gname=TABLE_NAME, _type=WaypointTask,
                                 action_points_dict={
                                     wp_g.name: WayFrame(
                                         wp_g.name, wp_g, [0, 0, wp_g.dims[2] / 2], [0, 0, 0])})

## Clock

In [10]:
CLOCK_NAME = "clock"
CLOCK_DIM = (0.14,0.05,0.076)
clock_center = np.add(wp_g.center[:2]+(0,), (0,0.1,TABLE_DIM[2]+CLOCK_DIM[2]/2+CLEARANCE))

In [11]:
clock_g = gscene.create_safe(GEOTYPE.BOX, CLOCK_NAME, link_name="base_link",dims=CLOCK_DIM,
                           center=clock_center, rpy=(0,0,0),
                           fixed=False, collision=True, color=(0.8,0.5,0.3,1))

push_point_list = []
for i in range(4):
    R = Rot_axis_series([3, 2], [np.pi/2*i, np.pi/2])
    point = np.round(np.multiply(CLOCK_DIM, -R[:,2])/2, 4)
    point[2] = -CLOCK_DIM[2]/2 + TOOL_PUSH_HEIGHT
    ap_name = "side{}".format(i)
    push_point_list.append(FramePoint(ap_name, clock_g, point=point, rpy=Rot2rpy(R), key=TOOL_PUSH_KEY))

slide_point = SlidePoint("bottom_p", clock_g, point=(0,0,-CLOCK_DIM[2]/2-CLEARANCE), rpy=(0,0,0), 
                         binded_on=table_b, dist_push=0.2, dir_push=[-1,0,0], key=TOOL_PUSH_KEY)

clock_s = pscene.create_subject(oname=CLOCK_NAME, gname=CLOCK_NAME, _type=PushObject, 
                              push_point_list=push_point_list, slide_point=slide_point)

## Bag

In [12]:
BAG_NAME = "bag"
BAG_DIM = (0.4, 0.29,0.64)

In [13]:
bag_g = gscene.create_safe(GEOTYPE.BOX, BAG_NAME, link_name="base_link",dims=BAG_DIM,
                           center=(1,0,BAG_DIM[2]/2), rpy=(0,0,0),
                           fixed=False, collision=True, color=(0.8,0.8,0.8,1))
handle = gscene.create_safe(GEOTYPE.BOX, BAG_NAME+"_hdl", link_name="base_link",dims=(0.2,0.03,0.02),
                           center=(0,0,BAG_DIM[2]/2+0.01), rpy=(0,0,0),
                           fixed=False, collision=True, color=(0.6,0.6,0.6,1), parent=BAG_NAME)

push_point_list = []
for i in range(4):
    R = Rot_axis_series([3, 2], [np.pi/2*i, np.pi/2])
    point = np.round(np.multiply(BAG_DIM, -R[:,2])/2, 4)
    point[2] = -BAG_DIM[2]/2 + MOBILE_PUSH_HEIGHT
    ap_name = "side{}".format(i)
    push_point_list.append(FramePoint(ap_name, bag_g, point=point, rpy=Rot2rpy(R), key=MOBILE_PUSH_KEY))

slide_point = SlidePoint("bottom_p", bag_g, point=(0,0,-BAG_DIM[2]/2-CLEARANCE), rpy=(0,0,0), 
                                           binded_on=floor_b, dist_push=1.0, key=MOBILE_PUSH_KEY)

bag_s = pscene.create_subject(oname=BAG_NAME, gname=BAG_NAME, _type=PushObject, 
                              push_point_list=push_point_list, slide_point=slide_point)

### prepare planner

In [14]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=1000, node_count_max=100, allow_joint_motion=True)
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
mplan.motion_filters = [gcheck, rcheck]
mplan.incremental_constraint_motion = True

ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
[WARN] reach data is not ready for kmb. Ignoring this robot...


### Custom rules

In [15]:
# joint motion is not allowed when pushing object with holder
idc_push = np.where([stype == PushObject for stype in pscene.subject_type_list])[0]
tplan.explicit_rule = lambda pscene, node, leaf: not any([node[idx]==leaf[idx] and "push" in node[idx] for idx in idc_push])

In [16]:
# generate base pose to reach targets
N_samples = 50
thetas_available = np.array([np.pi/2*i for i in range(4)])

xy_list = floor_b.geometry.get_vertice_radius_from(crob.home_dict)[0][:,:2]
xy_max = np.max(xy_list,axis=0)
xy_min = np.min(xy_list,axis=0)

targets = [pscene.subject_dict[sname] for sname, stype in zip(pscene.subject_name_list, pscene.subject_type_list) if stype == WaypointTask]
wps_dict = {target.oname: [target.action_points_dict[wname] for wname in target.action_points_order] for target in targets}
def gen_reaching_base(state):
    xyt_samples = np.array([np.random.uniform(xy_min, xy_max).tolist()
                            +[np.random.choice(thetas_available)] 
                            for _ in range(N_samples)])

    # check availability for wps
    Twe = np.linalg.inv(brush_b.Toff_lh)
    Tbe_mat = []
    for target in targets:
        i_s = pscene.subject_name_list.index(target.oname)
        ntem = state.node[i_s]
        wps = wps_dict[target.oname][ntem:] # exclude already reached waypoints
        Tbe_arr = []
        for wp in wps:
            Tbe = np.matmul(wp.get_tf_handle(crob.home_pose), Twe)
            Tbe_arr.append(Tbe)
        Tbe_mat.append(Tbe_arr)
        
    # score with more weights on reachable num on same targets
    score_mat = []
    for i_x, xyt in enumerate(xyt_samples):
        Tbr = gscene.get_tf(ROBOT_BASE, [xyt[0], xyt[1], xyt[2]]+[0]*9)
        score_list = []
        for Tbe_arr in Tbe_mat: # iterate targets
            score_t = 0
            for Tbe in Tbe_arr: # iterate waypoints
                Tre = np.matmul(np.linalg.inv(Tbr), Tbe)
                res = rcheck.check_T(ROBOT_NAME, Tre)
                if res:
                    Q = np.copy(state.Q)
                    Q[:3] = xyt
                    res = mplan.validate_trajectory([Q])
                    score_t += res
            score_list.append(score_t)
        score_mat.append(score_list)
    score_mat = np.square(score_mat) # (pose_num, target_num), square to weight more on large counts
    score_list = np.sum(score_mat, axis=-1,dtype=float)
    score_sum = np.sum(score_list)
    if score_sum>0:
        score_list = np.divide(score_list, score_sum)
        i_select = np.random.choice(range(N_samples), p=score_list)
        Q = np.copy(state.Q)
        Q[:3] = xyt_samples[i_select]
        return Q
    return None

tplan.config_gen = gen_reaching_base

### Plan

In [17]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
gscene.update_markers_all()
print(pscene.subject_name_list)
print(initial_state.node)

['bag', 'clock', 'waypoints']
('floor_ws', 'table', 0)


In [22]:
goal_nodes = [('floor_ws', 'table', 1)]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=100, 
              multiprocess=True, add_homing=True)

Use 18/36 agents
try: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
try: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 0 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
try: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
error: 2.3
Motion Filer Failure: GraspChecker
try transition motion
result: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
Motion Filer Failure: GraspChecker
result: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 0 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
try: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
error: 2.3
result: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0) = success
branching: 0->1 (0.25/100.0 s, s

error: 2.3
transition motion tried: True
try transition motion
branching: 2->9 (0.87/100.0 s, steps/err: 2(8.20088386536 ms)/0)
result: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 8 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
try: 3 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
try: 3 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: ReachChecker
Motion Filer Failure: GraspChecker
try: 4 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
result: 3 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
result: 7 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
transition motion tried: False
result: 9 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
branching: 3->10 (0.9/100.0 s, steps/err: 16(383.671045303 ms)/0.00164881984466)
try: 10 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
error: 0.35
Motion Filer Failure: GraspChecker
result: 3 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fai

result: 6 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 11 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
Motion Filer Failure: GraspChecker
branching: 4->18 (1.12/100.0 s, steps/err: 2(4.31609153748 ms)/0)
try: 18 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
try: 8 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
result: 4 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
try: 15 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
try: 10 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
try: 18 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
try: 1 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
Motion Filer Failure: GraspChecker
Motion Filer Failure: GraspChecker
result: 15 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0) = success
try: 19 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
result: 10 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = fail
transition motion tried: True
Motion Filer Failure: GraspChecker
Motion F

try: 25 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
transition motion tried: True
Motion Filer Failure: ReachChecker
branching: 1->25 (1.36/100.0 s, steps/err: 17(360.439062119 ms)/0.00108681436138)
result: 10 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = fail
try: 21 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
error: 0.7
try: 19 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: GraspChecker
transition motion tried: True
transition motion tried: True
result: 11 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 0 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
result: 3 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
result: 9 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
transition motion tried: False
Motion Filer Failure: GraspChecker
Motion Filer Failure: GraspChecker
Motion Plan Failure
result: 19 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
branching: 3->26 (1.39/100.0 s, steps/e

Motion Filer Failure: GraspChecker
Motion Filer Failure: ReachChecker
try: 7 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
branching: 12->35 (1.64/100.0 s, steps/err: 2(4.43196296692 ms)/0)
transition motion tried: True
try: 8 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
Motion Filer Failure: GraspChecker
Motion Filer Failure: GraspChecker
result: 34 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = fail
transition motion tried: True
result: 20 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
try: 36 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
try: 10 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
try: 34 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
result: 16 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
Motion Filer Failure: ReachChecker
result: 19 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
try: 37 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
Motion Filer Failure: GraspChecker
error: 0.064
branching: 20->36

...

try: 40 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
result: 62 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
++ adding return motion to acquired answer ++
try: 83 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
result: 167 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
result: 36 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = fail
result: 121 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
error: 0.61
try: 39 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
try: 146 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
Motion Filer Failure: GraspChecker
try: 121 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
error: 2.3
try transition motion
try: 2 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
result: 39 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
error: 2.2
try transition motion
try: 106 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
Motion Filer Failure: ReachChecker
try: 54 - ('floor_ws', 'table', 0)->('floor_

try: 54 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
result: 39 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
Motion Filer Failure: GraspChecker
try: 139 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
Motion Filer Failure: GraspChecker
try: 37 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
result: 96 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
result: 59 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
Motion Filer Failure: GraspChecker
error: 2.3
result: 69 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
try: 72 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
try transition motion
try: 7 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
Motion Filer Failure: ReachChecker
try: 139 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
result: 37 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = fail
try: 149 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
error: 0.0017
result: 54 - ('floor_ws', 'table', 0)->('floor_ws', '

branching: 72->188 (6.24/100.0 s, steps/err: 2(225.227117538 ms)/0.00133932884206)
Motion Plan Failure
transition motion tried: True
try: 6 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
result: 121 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 106 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
try: 188 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
try: 174 - ('floor_ws', 'table', 0)->('mpush', 'table', 0)
Motion Filer Failure: ReachChecker
try: 82 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
try: 61 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
result: 121 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
result: 121 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = fail
Motion Filer Failure: GraspChecker
branching: 121->189 (6.26/100.0 s, steps/err: 20(513.015985489 ms)/0.00140586541873)
Motion Filer Failure: GraspChecker
error: 0.35
result: 61 - ('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0) = fail
try: 189 - ('mpush'

branching: 7->197 (6.49/100.0 s, steps/err: 18(468.632221222 ms)/0.00117559749495)
transition motion tried: True
Goal reached
transition motion tried: True
Motion Filer Failure: GraspChecker
try: 196 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
try: 197 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
try: 199 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
result: 149 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
branching: 149->199 (6.52/100.0 s, steps/err: 18(225.117921829 ms)/0.00156336299533)
result: 20 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
branching: 20->200 (6.53/100.0 s, steps/err: 3(74.4559764862 ms)/0.00148613755751)
result: 195 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = fail
Motion Filer Failure: GraspChecker
Motion Filer Failure: GraspChecker
try: 164 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
try: 200 - ('mpush', 'table', 0)->('floor_ws', 'table', 0)
result: 199 - ('mpush', 'table', 0)->('floor_ws', 'table',

result: 1 - ('floor_ws', 'table', 0)->('floor_ws', 'table', 1) = fail
======================= terminated 2: Stop called from other agent  (6.8/100.0) ===============================
transition motion tried: True
======================= terminated 14: Stop called from other agent  (6.8/100.0) ===============================
======================= terminated 9: Stop called from other agent  (6.8/100.0) ===============================
result: 65 - ('floor_ws', 'table', 0)->('mpush', 'table', 0) = success
end
branching: 65->210 (6.76/100.0 s, steps/err: 2(546.471834183 ms)/0.00163827543739)
constrained motion tried: True
result: 207 - ('mpush', 'table', 0)->('floor_ws', 'table', 0) = success
======================= terminated 8: Stop called from other agent  (6.8/100.0) ===============================
branching: 207->211 (6.79/100.0 s, steps/err: 8(80.4750919342 ms)/1.5578664962e-07)
======================= terminated 7: Stop called from other agent  (6.8/100.0) ==========================

In [23]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

In [24]:
snode_schedule = tplan.get_best_schedule(at_home=False)
ppline.play_schedule(snode_schedule)

('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('mpush', 'table', 0)
('mpush', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('mpush', 'table', 0)
('mpush', 'table', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 'tpush', 0)
('floor_ws', 'tpush', 0)->('floor_ws', 'table', 0)
('floor_ws', 'table', 0)->('floor_ws', 'table', 1)
('floor_ws', 'table', 1)->('floor_ws', 'table', 1)
('floor_ws', 'table', 1)->('floor_ws', 'table', 1)
